# Introduction

In this tutorial we will deploy a more advanced version of the temperature converter we created in the "Simple Temperature Converter tutorial". It will support arbitrary conversion between Celsius, Fahrenheit and Kelvin of arrays of temperature values. Of course, having a model for such a simple task would usually be overkill, but it's a nice example. Deploying a notebook is great for simple models and trying out things. For more complex models with a lot of code and many dependencies you should consider creating a proper source package.

**Requirements**:
- You need to have the `cognite-model-hosting-notebook` package installed
- The environment variable `COGNITE_API_KEY` should be set

# Defining our model

First off we need to define our dependencies. This is simply achieved by creating a code cell that start with `# !requirements`. Our model will use `numpy` for calculations on arrays of numbers, so we'll be sure to add that. It's always good practice to be specific of the version of the package to avoid breaking changes in the future.

In [1]:
# !requirements
# numpy==1.16.2

Then we need to specify the code the prediction routine of our model will run.

In [2]:
# !model
import numpy as np

In [3]:
# !model
def celsius_to_kelvin(t):
    return t + 273.15

def fahrenheit_to_kelvin(t):
    return (t + 459.67) * (5/9)

def kelvin_to_celsius(t):
    return t - 273.15

def kelvin_to_fahrenheit(t):
    return t * (9/5) - 459.67

def to_kelvin(t, from_scale):
    if from_scale == "K":
        return t
    elif from_scale == "C":
        return celsius_to_kelvin(t)
    elif from_scale == "F":
        return fahrenheit_to_kelvin(t)
    else:
        raise AssertionError("Don't recognize temperature scale `{}`".format(from_scale))

def from_kelvin(t, to_scale):
    if to_scale == "K":
        return t
    elif to_scale == "C":
        return kelvin_to_celsius(t)
    elif to_scale == "F":
        return kelvin_to_fahrenheit(t)
    else:
        raise AssertionError("Don't recognize temperature scale `{}`".format(to_scale))

In [4]:
# !model
def predict(instance, from_scale, to_scale):
    t = np.array(instance)
    
    t = to_kelvin(t, from_scale)
    t = from_kelvin(t, to_scale)
    
    return t.tolist() # The returned value must be JSON serializable

Notice that we start each cell with `# !model`. This means this code will be included in our model. The next cells below will only be for local execution, and are not supposed to be executed in Model Hosting. And thus do not start with `# !model`. For our model to work with Model Hosting we have to define a `predict()` function. It's first parameter must be `instance` - the entity on which to perform prediction on. After that we may define our own parameters. We can of course also define other functions (and classes, etc) to help us do the prediction as we have done above.

Okay, now we have defined our model. Let's try it out locally before we deploy it to Model Hosting.

In [5]:
predict([70, 80, 90], from_scale="F", to_scale="C")

[21.1111111111112, 26.666666666666742, 32.222222222222285]

In [6]:
predict([0, -10, 10], from_scale="C", to_scale="F")

[31.999999999999943, 13.999999999999943, 49.99999999999994]

In [7]:
predict([273.15, 373.15, 300], from_scale="K", to_scale="C")

[0.0, 100.0, 26.850000000000023]

Seems to be working nicely!

# Deploying our model

It's time to deploy our model. Let's first import what we need.

In [8]:
from cognite.client.experimental import CogniteClient
from cognite.model_hosting.notebook import deploy_model_version

model_hosting = CogniteClient().model_hosting

To deploy a model version we first need a model.

In [9]:
model_name = "temperature-converter"
model_hosting.models.create_model(model_name)

And then we deploy this notebook as a model version. It's important you **save the notebook** before doing this since the notebook file will be read to find your model.

In [10]:
model_version_name = "temperature-converter-v1"
deploy_model_version(
    model_name=model_name,
    version_name=model_version_name,
    runtime_version="0.1"
)

Now we just have to wait for the deployment to finish. This usually takes a few minutes. Notice that the next step won't work before the status of the model version is 'READY'.

In [11]:
model_hosting.versions.get_model_version(model_name, model_version_name).status

'READY'

# Testing the model

Now that it's deployed we can test it out.

In [12]:
model_hosting.models.online_predict(
    model_name,
    model_version_name,
    instances=[[0, -10, 10], [-50, 100]],
    args={"from_scale": "C", "to_scale": "F"}
)

[[31.999999999999943, 13.999999999999943, 49.99999999999994],
 [-58.00000000000006, 211.99999999999994]]

It's important to note that these predictions were performed in Cognite Model Hosting in the cloud, not on your computer. Anyone with internet (and appropriate access rights) can now access this model!

When you're done - remember to clean up after yourself:

In [13]:
source_package_id = model_hosting.versions.get_model_version(model_name, model_version_name).source_package_id
model_hosting.models.delete_model(model_name)
model_hosting.source_packages.delete_source_package(source_package_id)